In [57]:
import pandas as pd
import numpy as np
import finpy_tse as fpy
import datetime
import jdatetime as jdt
from CONFIG import DB_PASS as PASS
from metafid.mfw import DB


In [58]:
db = DB(dbname="metafid", user="postgres", pass_=PASS)

### Stock List

In [63]:
tickers = db.query_all(table="tsedata_ticker", cols="symbol, symbol_far,market_id")
tse_tickers= tickers[tickers.market_id==1]

### Update `tsedata_section` DB

In [67]:
def update_ri(stock_list):

    # get max date
    date_id = db.query_all(table="tsedata_histprice", cols="date_id")

    max_date = date_id.date_id.max()
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    date = db.query_where(table="tsedata_date", cols="date,jweekday", where=f" date <= '{today}' and date > '{max_date}'")
    date = date[date.jweekday.isin([0,1,2,3,4])]


    # take data an insert to DB
    if len(date):
        start_date = date.date[0]
        start_date = jdt.date.fromgregorian(year = start_date.year, day=start_date.day, month=start_date.month).strftime("%Y-%m-%d")
        df = pd.DataFrame()
        for stock in stock_list.symbol_far:
            df_ri = fpy.Get_RI_History(
                stock=stock,
                start_date=start_date,
                end_date=jdt.date.today().strftime("%Y-%m-%d"),
                ignore_date=False,
                show_weekday=False,
                double_date=True).rename(columns=str.lower).drop(["name","market"], axis=1)
            df= pd.concat([df, df_ri])
        df = df.merge(stock_list, left_on="ticker", right_on="symbol_far", how="inner")
        df.drop(["ticker","symbol_far", "market_id" ], axis=1, inplace=True)
        df.rename(columns={"date":"date_id", "symbol":"symbol_id"}, inplace=True)
        return db.insert_data(table="tsedata_section", df=df)
    return print("DB is update")
update_ri(tse_tickers)

### Update `tsedata_histprice` DB

In [70]:
def update_hist_price(stock_list):

    # get max date
    date_id = db.query_all(table="tsedata_histprice", cols="date_id")

    max_date = date_id.date_id.max()
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    date = db.query_where(table="tsedata_date", cols="date,jweekday", where=f" date <= '{today}' and date > '{max_date}'")
    date = date[date.jweekday.isin([0,1,2,3,4])]
    
    # take data an insert to DB
    if len(date):
        start_date = date.date[0]
        start_date = jdt.date.fromgregorian(year = start_date.year, day=start_date.day, month=start_date.month).strftime("%Y-%m-%d")
        df = pd.DataFrame()
        for stock in stock_list.symbol_far:
            df_hist = fpy.Get_Price_History(
                stock=stock,
                start_date=start_date,
                end_date=jdt.date.today().strftime("%Y-%m-%d"),
                ignore_date=False,
                adjust_price=True,
                show_weekday=False,
                double_date=True).rename(columns=str.lower).drop(["open","high","low", "close", "final", "no","name","market"], axis=1)
            df_hist.columns = [col.replace("adj ", "") for col in df_hist.columns]
            df= pd.concat([df, df_hist])
            
        df = df.merge(stock_list, left_on="ticker", right_on="symbol_far", how="inner")
        df.drop(["ticker","symbol_far", "market_id" ], axis=1, inplace=True)
        df.rename(columns={"date":"date_id", "symbol":"symbol_id"}, inplace=True)
        return db.insert_data(table="tsedata_histprice", df=df)
    return print("DB is update")
update_hist_price(tse_tickers)